In [1]:
import pandas as pd
import win32com.client as win32

from tkinter import *
import tkinter.filedialog

from tkinter import messagebox
root= Tk()
arquivo = tkinter.filedialog.askopenfilename(title = "Selecione o Arquivo csv com Canais e Keywords")
root.destroy()
busca_df  = pd.read_excel(arquivo)
display(busca_df)

,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service

servico = Service(r'C:\Users\usuario\anaconda3\chromedriver.exe')
navegador = webdriver.Chrome(service=servico)

In [3]:
def bus_produtos_shopping(navegador, produto, preco_minimo, preco_maximo) :

    
    navegador.get('https://www.google.com/')
    navegador.find_element(By.XPATH, '//*[@id="APjFqb"]').send_keys(produto)
    navegador.find_element(By.XPATH, '//*[@id="APjFqb"]').send_keys(Keys.ENTER)
    navegador.find_element(By.XPATH, '//*[@id="bqHHPb"]/div/div/div[1]/a[1]/div').click()


    lista_resultado = navegador.find_elements(By.CLASS_NAME, 'KZmu8e')
    lista_oferta = []
    for resultado in lista_resultado :
        nome = resultado.find_element(By.TAG_NAME, 'h3').text

        try :
            preco = resultado.find_element(By.CLASS_NAME, 'hn9kf').text
            preco = preco.replace('R$', '').replace('.', '').replace(',', '.')
            preco_float = float(preco)
            link = resultado.find_element(By.CLASS_NAME, 'shntl').get_attribute('href')

            if preco_minimo <= preco_float <= preco_maximo :
                lista_oferta.append((nome, preco, link))
                
        except :

            pass
    return lista_oferta



def bus_produtos_buscape(navegador, produto, preco_minimo, preco_maximo) :

    navegador.get('https://www.buscape.com.br/')
    navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').click()
    navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/input').send_keys(produto)

    navegador.find_element(By.XPATH, '//*[@id="new-header"]/div[1]/div/div/div[3]/div/div/div[2]/div/div[1]/button').click()
    navegador.execute_script("window.scrollTo(0, 400)")
    lista_resultado = navegador.find_elements(By.CLASS_NAME, 'ProductCard_ProductCard__WWKKW')


    lista_ofertas = []
    for resultado in lista_resultado :
        nome = resultado.find_element(By.TAG_NAME, 'h2').text

        try :
            preco = resultado.find_element(By.CLASS_NAME, 'Text_MobileHeadingS__HEz7L').text
            preco = preco.replace('R$', '').replace('.', '').replace(',', '.')
            preco_float = float(preco)
            link = resultado.find_element(By.CLASS_NAME, 'ProductCard_ProductCard_Inner__gapsh').get_attribute('href')

            if preco_float <= preco_minimo <= preco_maximo :
                lista_ofertas.append((nome, preco_float, link))

        except :
            pass
    return lista_ofertas






In [4]:
for linha in busca_df.index :
    produto = busca_df.loc[linha, 'Nome']
    preco_minimo = busca_df.loc[linha, 'Preço mínimo']
    preco_maximo = busca_df.loc[linha, 'Preço máximo']
    
    
    lista_busca_shopping = bus_produtos_shopping(navegador, produto, preco_minimo, preco_maximo)
    
    if lista_busca_shopping :
        tabela_shopping = pd.DataFrame(lista_busca_shopping, columns=['Nome do produto', 'Preço do produto', 'Link para compra'])
        
        outlook = win32.Dispatch('outlook.application')

        mail = outlook.CreateItem(0)
        mail.To = 'jeanlusjeanmarybresil@gmail.com'
        mail.Subject = 'Prezado Eng  Jean Mary Jeanlus Os produtos  mais baratos são :'
        mail.HTMLBody = f'''
        <p> Encontramos esses  produtos  na  faixa  do preçco desejado{tabela_shopping.to_html()} </p>
        '''

        mail.Send()
    else :
        lista_busca_shopping = None
        
        
    lista_busca_bucape = bus_produtos_buscape(navegador, produto, preco_minimo, preco_maximo)
    
    if lista_busca_bucape :
        tabela_buscape = pd.DataFrame(lista_busca_bucape, columns=['Nome do produto', 'Preço do produto', 'Link para compra'])
        
         
        outlook = win32.Dispatch('outlook.application')

        mail = outlook.CreateItem(0)
        mail.To = 'jeanlusjeanmarybresil@gmail.com'
        mail.Subject = 'Prezado Eng  Jean Mary Jeanlus Os produtos  mais baratos são :'
        mail.HTMLBody = f'''
        <p> Encontramos esses  produtos  na  faixa  do preçco desejado{tabela_buscape.to_html()} </p>
        '''

        mail.Send()
    else :
        lista_busca_shopping = None
    